1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
    토큰수 초과로 답변을 생성하지 못할수 있고
    문서가 길면(인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [21]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter)

In [22]:
len(document_list)

187

In [31]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')



In [35]:
from langchain_community.vectorstores import Chroma

# chroma는 인메모리이기에 디렉토리에 저장을 하자
#database = Chroma.from_documents(document_list, embedding=embedding, collection_name="choroma-tax", persist_directory="./chroma")
database = Chroma(collection_name='chroma-tax', persist_directory='./chroma', embedding=embedding)


TypeError: Chroma.__init__() got an unexpected keyword argument 'embedding'

In [10]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query)

ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/guides/embeddings in query.

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')


In [13]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrived_docs}

Question: {query}
"""

NameError: name 'retrived_docs' is not defined

In [ ]:
ai_message = llm.invoke(prompt)

In [ ]:
ai_message.content

In [26]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

d:\00-git\LLMS\rag\env\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [27]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [28]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [36]:
ai_message = qa_chain({"query":query} )

C:\Users\son\AppData\Local\Temp\ipykernel_24336\1097441378.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query":query} )


ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/guides/embeddings in query.